In [ ]:
import xarray as xr
import dask.array as da
import numpy as np
from blockwise_stack import blockwise_stack


one_to_six = np.array([[1, 2, 3], [4, 5, 6]])


ds = xr.Dataset(
    {
        "data": (
            ["y", "x", "band"],
            da.random.random((4, 6, 3), chunks=(2, 2, -1)),
        )
    },
    coords={
        "chunk_idx": (
            ("y", "x"),
            np.repeat(np.repeat(one_to_six, 2, axis=0), 2, axis=1),
        )
    },
)
ds.data

In [ ]:
ds.data.data.visualize()

In [ ]:
ds.chunk_idx

In [ ]:
ds.data.chunks

In [ ]:
stacked = ds.stack(pixel=("y", "x"))
stacked.data

In [ ]:
stacked.data.data.visualize()

In [ ]:
stacked.chunk_idx

In [ ]:
stacked.data.chunks

In [ ]:
t = ds.copy()
t.coords["chunk_idx"] = (("y", "x"), da.from_array(t.chunk_idx.values, chunks=(2, 2)))

stacked_blockwise = blockwise_stack(
    t,
    input_dims=("y", "x"),
    output_dim="pixel",
    ds_chunks={"y": 2, "x": 2, "band": -1},
)
stacked_blockwise.data

In [ ]:
stacked_blockwise.data.data.visualize()

In [ ]:
stacked_blockwise.chunk_idx.compute()

In [ ]:
stacked_blockwise.data.chunks